In this notebook, we will be preparing anything that might be needed for future steps, such as co-expression networks and integrating different data sources (both from the challenge and from elsewhere).

### Co-Expression Network

--- WRITE SOME INTRO! --- # TODO

In [60]:
import os
os.getcwd()

ModuleNotFoundError: No module named 'matplotlib'

In [1]:
import dask.dataframe as dd
import pandas as pd

target_cell = "B cells"

# multiome_obs_meta = pd.read_csv("../data/multiome_obs_meta.csv")
# obs_ids = multiome_obs_meta[multiome_obs_meta["cell_type"] == target_cell]["obs_id"].to_list()

# multiome_var_meta = pd.read_csv("../data/multiome_var_meta.csv")
# gene_names = multiome_var_meta[multiome_var_meta["feature_type"] == "Gene Expression"]["location"].to_list()

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [2]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.metadata.num_rows // batch_num
for batch in adata_file.iter_batches(batch_size=batch_group_size):
    adata_batch = batch.to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

In [12]:
target_adata_df

,obs_id,gene,count,normalized_count
0,000683e13aa73f7f,A2M-AS1,1,5.736340
1,000683e13aa73f7f,ABCB8,1,5.736340
2,000683e13aa73f7f,ABCE1,2,6.427873
3,000683e13aa73f7f,ABI1,1,5.736340
4,000683e13aa73f7f,ABT1,1,5.736340
...,...,...,...,...
58266296,fffcb76d7d937647,ZSWIM7,2,4.384153
58266297,fffcb76d7d937647,ZUP1,1,3.703402
58266298,fffcb76d7d937647,ZWILCH,2,4.384153
58266299,fffcb76d7d937647,ZXDC,1,3.703402


In [22]:
target_data_df = target_adata_df.pivot_table("normalized_count", "obs_id", "gene")
target_data_df

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,NaN,NaN,NaN,5.736340,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0008106bb52a931d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000a41a4580023f5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.238042,NaN
000a59643eff306a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.921865,...,NaN,NaN,4.921865,NaN,NaN,NaN,NaN,NaN,5.611362,NaN
000a7dc7f1cba897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.240418,...,4.240418,NaN,NaN,NaN,NaN,4.240418,NaN,4.240418,4.240418,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6f64b0cd2160b,NaN,5.212115,NaN,5.212115,NaN,NaN,NaN,NaN,NaN,5.902534,...,NaN,NaN,NaN,NaN,NaN,5.212115,NaN,NaN,NaN,NaN
fff97cc3be63ebc0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.239318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffb8bd514c27106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
adata_obs_meta[adata_obs_meta['col']==3]

,obs_id,library_id,plate_name,well,row,col,cell_id,donor_id,cell_type,sm_lincs_id,sm_name,SMILES,dose_uM,timepoint_hr,control
19,000683e13aa73f7f,library_44,plate_1,A3,A,3,PBMC,donor_2,T cells CD8+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
23,0008106bb52a931d,library_19,plate_2,H3,H,3,PBMC,donor_0,B cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
37,000a41a4580023f5,library_43,plate_2,G3,G,3,PBMC,donor_0,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
38,000a59643eff306a,library_47,plate_2,F3,F,3,PBMC,donor_0,T cells CD4+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
39,000a7dc7f1cba897,library_7,plate_4,H3,H,3,PBMC,donor_2,Myeloid cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240045,fff6f64b0cd2160b,library_24,plate_5,G3,G,3,PBMC,donor_1,T cells CD8+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240055,fff97cc3be63ebc0,library_31,plate_4,E3,E,3,PBMC,donor_2,NK cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240067,fffb8bd514c27106,library_45,plate_5,A3,A,3,PBMC,donor_1,B cells,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True
240073,fffc885a62e5dab7,library_27,plate_0,G3,G,3,PBMC,donor_0,T cells CD4+,LSM-36361,Dimethyl Sulfoxide,C[S+](C)[O-],14.1,24,True


In [17]:
pd.DataFrame(adata_obs_meta[adata_obs_meta['col']==3].groupby(
    ["plate_name", "well", "cell_type"])["obs_id"].apply(list)).to_csv("../config/obs_id_groups.csv")

combine observations gene expression for each cell type

In [18]:
exp_groups = pd.read_csv("../config/obs_id_groups.csv")
B_cells = exp_groups[exp_groups["cell_type"]=="B cells"]
Myeloid_cells = exp_groups[exp_groups["cell_type"]=="Myeloid cells"]
NK_cells = exp_groups[exp_groups["cell_type"]=="NK cells"]
T_cells_CD4 = exp_groups[exp_groups["cell_type"]=="T cells CD4+"]
T_cells_CD8 = exp_groups[exp_groups["cell_type"]=="T cells CD8+"]
T_regulatory_cells = exp_groups[exp_groups["cell_type"]=="T regulatory cells"]

In [31]:
B_cells.head(3)

,plate_name,well,cell_type,obs_id
0,plate_0,A3,B cells,"['010facc6f76220ca', '035f4d18aa398e07', '08a4..."
6,plate_0,B3,B cells,"['048b7f9c92520bd4', '068c88fe72f5faaf', '0869..."
12,plate_0,C3,B cells,"['03769b79dc8b5fdb', '03772c380a73b43d', '05a7..."


In [50]:
def create_combined_obs_exp(groups, exp_data):
    cell_type_exp = pd.DataFrame()
    for index, row in enumerate(groups["obs_id"]):
        row = eval(row)
        sample = exp_data.loc[row]
        max_exp_df = pd.DataFrame(sample.max())
        column_name = "_".join(list(groups[["plate_name", "well", "cell_type"]].iloc[index]))
        max_exp_df = max_exp_df.rename(columns = {0:column_name})
        cell_type_exp = pd.concat([cell_type_exp, max_exp_df], axis=1)
    return cell_type_exp
        

In [25]:
combined_df = pd.DataFrame()
cell_types = [B_cells, Myeloid_cells, T_cells_CD4, T_cells_CD8, NK_cells, T_regulatory_cells]
for cell_type in cell_types:
    combined_df = pd.concat(combined_df, create_combined_obs_exp(B_cells, target_data_df)

['010facc6f76220ca', '035f4d18aa398e07', '08a4e170b8995d47', '0c01ec2dbc90bf87', '0c26ba6f8fef4ccd', '0dad4643fc429089', '0f61ec7552ef2473', '0ffcbf6ea14b0fe8', '1038f1aa9faa4760', '10f8ec2701d14af2', '117469622007b7f0', '1277f11ff350933e', '14b35759daa18adb', '15977791bfe6002c', '15c4f2d03ae34413', '161b49f6a4acc0ac', '1c36a439e2a746d5', '1e380b839aeb409f', '220855642a2e79fb', '235de44fde5d9887', '243ab6be3c60892c', '2679767329ec78f1', '26c57b3636afe7a8', '2a0bb9ea40ef62b1', '2a2ad34b19bd8b2e', '2ad3669e06d47a4d', '30f536e23852f68c', '32d52c98852cb24f', '34b7cdea9a14215b', '367be8cab4dc8664', '387183c64db8c1e1', '38a5bfd3bba87573', '3c9163fb1464c445', '3e0441a4a6ab77f2', '3ead234a406f67f1', '41ba562ed33eda37', '42558ba0e29175e9', '45ba76da8f4087f2', '4676e93d1599008b', '4902ecdd4153a038', '4a99ebdfd613a3e5', '4b8be26492c14ab6', '503a174f2be0f417', '507cd6f58bf4bf48', '50dbb449f45d8edc', '52c80cb1fc7f795d', '566f3ebfbbd67b87', '57a65133f748299e', '59a1e05ed7ce01d0', '5ab677c9e8acad0f',

count    12732.000000
mean         5.744409
std          0.744875
min          3.601916
25%          5.433714
50%          5.625996
75%          6.180589
max         13.067540
dtype: float64

'plate_0_A3_B cells'

In [63]:
create_combined_obs_exp(B_cells, target_data_df).dropna(axis=0).to_csv("../data/temp/B_cells_control_normalized.csv")
create_combined_obs_exp(T_cells_CD4, target_data_df).dropna(axis=0).to_csv("../data/temp/T_cells_CD4_control_normalized.csv")
create_combined_obs_exp(T_cells_CD8, target_data_df).dropna(axis=0).to_csv("../data/temp/T_cells_CD8_control_normalized.csv")
create_combined_obs_exp(Myeloid_cells, target_data_df).dropna(axis=0).to_csv("../data/temp/Myeloid_cells_control_normalized.csv")
create_combined_obs_exp(NK_cells, target_data_df).dropna(axis=0).to_csv("../data/temp/NK_cells_control_normalized.csv")
create_combined_obs_exp(T_regulatory_cells, target_data_df).dropna(axis=0).to_csv("../data/temp/T_regulatory_cells_control_normalized.csv")

In [64]:
B_cells_control = pd.read_csv("../data/temp/B_cells_control_normalized.csv")
T_cells_CD4_control = pd.read_csv("../data/temp/T_cells_CD4_control_normalized.csv")
T_cells_CD8_control = pd.read_csv("../data/temp/T_cells_CD8_control_normalized.csv")
Myeloid_cells_control = pd.read_csv("../data/temp/Myeloid_cells_control_normalized.csv")
NK_cells_control = pd.read_csv("../data/temp/NK_cells_control_normalized.csv")
T_regulatory_cells_control = pd.read_csv("../data/temp/T_regulatory_cells_control_normalized.csv")


In [83]:
B_cells_genes = list(B_cells_control["gene"])
common_genes = [i for i in B_cells_genes if i in list(T_cells_CD4_control["gene"])
                and i in list(T_cells_CD8_control["gene"])
                and i in list(Myeloid_cells_control["gene"])
                and i in list(NK_cells_control["gene"])
                and i in list(T_regulatory_cells_control["gene"])]

In [84]:
len(common_genes)

1118

In [12]:
target_data_df.max().describe()

count    21122.000000
mean         6.510980
std          0.724143
min          3.581188
25%          6.005600
50%          6.502292
75%          6.893881
max         13.438772
dtype: float64

In [24]:
import PyWGCNA
exp = PyWGCNA.WGCNA(name=target_cell, species="Homo sapiens", geneExp=target_data_df.fillna(0))
exp.geneExpr.to_df().head(5)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.0,0.0,5.73634,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.921865,...,0.000000,0.0,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,4.240418,...,4.240418,0.0,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0


In [25]:
exp.preprocess()

Pre-processing...
	Detecting genes and samples with too many missing values...


In [ ]:
exp.geneExpr.to_df().head(5)

In [2]:
from pyarrow.parquet import ParquetFile

target_adata_df = pd.DataFrame()
batch_num = 10

adata_file = ParquetFile("../data/adata_train.parquet")
batch_group_size = adata_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, adata_file.num_row_groups)
    adata_batch = adata_file.read_row_groups(s).to_pandas()
    adata_batch = adata_batch[adata_batch["obs_id"].isin(obs_ids)]
    target_adata_df = pd.concat([target_adata_df, adata_batch], axis=0, ignore_index=True)

In [23]:
target_data_df.notna().sum(axis=1)

obs_id
000683e13aa73f7f    1204
0008106bb52a931d    1052
000a41a4580023f5    1927
000a59643eff306a    1991
000a7dc7f1cba897    3580
                    ... 
fff6f64b0cd2160b    1609
fff97cc3be63ebc0    1355
fffb8bd514c27106    2382
fffc885a62e5dab7    2244
fffcb76d7d937647    4568
Length: 28462, dtype: int64

416442312

In [46]:
from pyarrow.parquet import ParquetFile

target_multiome_df = pd.DataFrame()
batch_num = 10

multiome_file = ParquetFile("../data/multiome_train.parquet")
batch_group_size = multiome_file.num_row_groups // batch_num
for batch in range(batch_num):
    if batch < batch_num - 1:
        s = range(batch*batch_group_size, (batch+1)*batch_group_size)
    else:
        s = range(batch*batch_group_size, multiome_file.num_row_groups)
    multiome_batch = multiome_file.read_row_groups(s).to_pandas()
    multiome_batch = multiome_batch[multiome_batch["obs_id"].isin(obs_ids)]
    # multiome_batch = multiome_batch[multiome_batch["location"].isin(gene_names)]
    target_multiome_df = pd.concat([target_multiome_df, multiome_batch], axis=0, ignore_index=True)

In [81]:
target_cell = "B cells"

adata_obs_meta = pd.read_csv("../data/adata_obs_meta.csv")
obs_ids = adata_obs_meta[adata_obs_meta["col"] == 3]["obs_id"].to_list()

In [88]:
x.fillna(0)

gene,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
obs_id,,,,,,,,,,,,,,,,,,,,,
000683e13aa73f7f,0.0,0.000000,0.000000,5.736340,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
0008106bb52a931d,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
000a41a4580023f5,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,5.238042,0.0
000a59643eff306a,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.921865,...,0.000000,0.000000,4.921865,0.0,0.0,0.000000,0.0,0.000000,5.611362,0.0
000a7dc7f1cba897,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.240418,...,4.240418,0.000000,0.000000,0.0,0.0,4.240418,0.0,4.240418,4.240418,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff6f64b0cd2160b,0.0,5.212115,0.000000,5.212115,0.0,0.0,0.000000,0.0,0.0,5.902534,...,0.000000,0.000000,0.000000,0.0,0.0,5.212115,0.0,0.000000,0.000000,0.0
fff97cc3be63ebc0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,6.239318,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
fffb8bd514c27106,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [91]:
c = x[1:10].corr()
c

MemoryError: Unable to allocate 3.32 GiB for an array with shape (21122, 21122) and data type float64

In [53]:
# Transform dataframe --> columns as genes and rows as samples...
x = target_multiome_df[target_multiome_df["location"].isin(gene_names)].pivot_table("normalized_count", "obs_id", "location")

In [86]:
target_multiome_df

,obs_id,location,count,normalized_count
0,000225c1151ab841,AAMP,1,6.320659
1,000225c1151ab841,AASS,1,6.320659
2,000225c1151ab841,ABCC11,1,6.320659
3,000225c1151ab841,ABCC2,1,6.320659
4,000225c1151ab841,ABR,1,6.320659
...,...,...,...,...
34311166,ffe93b989e73959e,chrX:63754752-63755597,1,2.770587
34311167,ffe93b989e73959e,chrX:64205465-64206345,2,3.048007
34311168,ffe93b989e73959e,chrX:6738626-6739524,2,8.954666
34311169,ffe93b989e73959e,chrX:78754674-78755560,2,9.734200


In [70]:
(x["A1BG"].notna() & x["ZXDC"].notna()).sum()

51